In [2]:
# 1. 필요 모듈 가져오기
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
from time import sleep
import re

In [5]:

# 직무이름 크롤링
def get_job_list():
    driver.get(f"https://www.saramin.co.kr/zf_user/jobs/list/job-category")
    # IT분야 클릭
    button_open = driver.find_element(By.XPATH, '//*[@id="sp_main_wrapper"]/div[2]/div/div[1]/div[2]/div[1]/button[6]')
    button_open.click()
    # 사람인의 경우 직무,직업/전문분야/기술스택 세가지 expand 버튼이 있음.
    # expand버튼 눌러서 카테고리 펼치기
    # 직업(job list)만 일단 추출
    xpath_list = [
        '//*[@id="sp_job_category_subDepth_2"]/div[2]/div/div[2]/div/dl[1]/dt/button',
        # '//*[@id="sp_job_category_subDepth_2"]/div[2]/div/div[2]/div/dl[2]/dt/button',
        # '//*[@id="sp_job_category_subDepth_2"]/div[2]/div/div[2]/div/dl[3]/dt/button',
    ]
    for xpath in xpath_list:
        # 해당 XPath를 가진 요소 찾기
        element = driver.find_element(By.XPATH, xpath)
        # 찾은 요소 클릭
        element.click()
        # 각 클릭 사이에 일정 시간을 주어 페이지 로딩을 기다림
        sleep(2)
    # 직무 리스트를 얻어내는 단계
    job_list = []
    jobs = driver.find_elements(By.XPATH, '//*[@id="sp_job_category_subDepth_2"]/div[2]/div/div[2]/div/dl[1]/dd/button')
    for job in jobs:
        job_list.append(job.text)
    driver.quit()
    return job_list



In [69]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
saramin_job_list = get_job_list()
print(saramin_job_list)

In [ ]:
saramin_job_list = [re.sub(r'\(.*?\)', '', job) for job in saramin_job_list]
print(saramin_job_list)

In [15]:
# 5. 직무별 채용공고 크롤링을 위해 job_list, job_id 정의
saramin_job_list = ['게임개발', '기술지원', '데이터분석가', '데이터엔지니어', '백엔드/서버개발', '보안관제',
                      '보안컨설팅', '앱개발', '웹개발', '웹마스터', '유지보수', '정보보안', 
                     '퍼블리셔', '프론트엔드', 'CISO', 'CPO', 'DBA', 'FAE', 'GM(게임운영)','ICT컨설팅',
                       'IT컨설팅','QA/테스터', 'SE(시스템엔지니어)', 'SI개발',  'SQA']

saramin_job_id = [80, 81, 82, 83, 84, 2239, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 102, 98, 99, 100, 101,2229]
saramin_jobs = dict(zip(saramin_job_id, saramin_job_list))

In [96]:

# Dataframe 생성
df = pd.DataFrame(columns=['직무명', '회사명', '채용공고명', '근무지역', '고용형태',
                            '경력조건', '교육', '마감일','출처'])  
''' job_name : 직무명
    company : 회사명
    jd_body : 채용공고상세
    head_title :채용공고명
    work_place : 근무지역 
    employment_type : 고용형태
    career : 경력조건
    education : 교육 
    deadlines : 마감일 
    jobs_url : 출처
    '''

' job_name : 직무명\n    company : 회사명\n    jd_body : 채용공고상세\n    head_title :채용공고명\n    work_place : 근무지역 \n    employment_type : 고용형태\n    career : 경력조건\n    education : 교육 \n    deadlines : 마감일 \n    jobs_url : 출처\n    '

In [95]:
# 직군별 채용공고 url, 및 기본 정보 수집
baseUrl = "https://www.saramin.co.kr"
jobs_url = []
def get_job_url(df, jobid):
    page = 1
    print(f'{saramin_jobs[jobid]}', '탐색 시작')
    while True:
        print(f'{page}page', '진행중')
        soup = requests.get('https://www.saramin.co.kr/zf_user/jobs/list/job-category?page={}&cat_kewd={}&search_optional_item=n&search_done=y&panel_count=y&preview=y&isAjaxRequest=0&page_count=50&sort=RL&type=job-category&is_param=1&isSearchResultEmpty=1&isSectionHome=0&searchParamCount=1#searchTitle'.format(page,jobid), headers={'User-Agent': 'Mozilla/5.0'})
        html = BeautifulSoup(soup.text, 'html.parser')
        jobs = html.select('div.list_item')

        if not jobs:  # jobs가 비어있으면 반복을 중단
            break

        for job in jobs:
            try:
                company = (job.select_one('a')['title'])
                job_url = baseUrl + job.select_one('a')['href']
                head_title = (job.select_one('div.job_tit > a')['title'])
                career = (job.select_one('p.career').text)
                education = (job.select_one('p.education').text)
                employment_type = (job.select_one('p.employment_type').text)
                work_place = (job.select_one('p.work_place').text)
                deadlines = (job.select_one('p.deadlines').text)
                new_row = {'직무명': saramin_jobs[jobid], '회사명': company,
                        '채용공고명': head_title, '근무지역': work_place,
                        '고용형태': employment_type, '경력조건': career, 
                        '교육': education,'마감일': deadlines,'출처': job_url}
                df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)
                jobs_url.append(job_url)  # URL을 jobs_url 리스트에 추가
                
            except Exception:
                pass

        page += 1  # 다음 페이지로 이동
    print(f'{saramin_jobs[jobid]}', '탐색 완료')
    return df

In [94]:
# frame안의 새로운 url찾기
frame_urls = []
def find_newurl(jobs_url):
    for job_url in tqdm(jobs_url):
        driver.get(job_url)
        driver.implicitly_wait(3)
        frame_urls.append(driver.find_element(By.XPATH, '//*[@id="iframe_content_0"]').get_attribute('src'))

In [128]:
def find_body(df, frame_urls):
    for i, frame_url in enumerate(tqdm(frame_urls)):
        try:
            driver.get(frame_url)
            driver.implicitly_wait(5)
            jd_body = driver.find_element(By.XPATH, '/html/body/div/div').text
            df.loc[i, '채용공고상세'] = jd_body
        except Exception:
            pass

In [99]:
# 직무별 채용공고 크롤링 시작
# 직무별 사이트로 들어가 직무별 채용공고 url 찾기
for jobid in tqdm(saramin_job_id):
    df = get_job_url(df,jobid)
# 셀레니움 실행
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
# print(f'{saramin_jobs[jobid]}', '탐색 완료')
# frame_url(채용공고 프레임 url 찾기)
find_newurl(jobs_url)
# frame_url에 들어가 채용공고 상세(jd_body) 찾기
find_body(df, frame_urls)
driver.quit()

  0%|          | 0/1 [00:00<?, ?it/s]

게임개발 탐색 시작
1page 진행중
2page 진행중
3page 진행중
4page 진행중
5page 진행중
6page 진행중
7page 진행중
8page 진행중
9page 진행중


100%|██████████| 1/1 [00:08<00:00,  8.41s/it]

게임개발 탐색 완료



100%|██████████| 386/386 [07:12<00:00,  1.12s/it]


In [120]:
df

,직무명,회사명,채용공고명,근무지역,고용형태,경력조건,교육,마감일,출처,채용공고상세
0,게임개발,(주)엔씨아이티에스,[NC 자회사] ㈜NC ITS ITSM 개발 인력 채용,경기 성남시 분당구,정규직,신입 · 경력,"대학(2,3년)↑",~ 06/28(수)(11일 전 등록),https://www.saramin.co.kr/zf_user/jobs/relay/v...,모집부문 및 자격요건\n모집부문 담당업무 상세 내용 인원\nFront-End\nDe...
1,게임개발,한국기술교육직업전문학교,"23년 하반기 전액국비 IT(JAVA/빅데이터, 클라우드/정보보안)취업교육생 모집",부산 동래구,교육생,경력무관,학력무관,~ 06/20(목)(3일 전 등록),https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[교육비무료]\n23년 하반기 전액국비 IT(JAVA/빅데이터,\n클라우드/정보보안..."
2,게임개발,(주)엔씨아이티에스,[NC 자회사] ㈜NC ITS 시스템 엔지니어 채용,경기 성남시 분당구,정규직 · 계약직,경력 5년↑,"대학(2,3년)↑",~ 06/29(목)(10일 전 등록),https://www.saramin.co.kr/zf_user/jobs/relay/v...,모집부문 및 자격요건\n모집부문 담당업무 상세 내용 인원\nNC IDC\n시스템 관...
3,게임개발,(주)엔씨아이티에스,[NC 자회사] ㈜NC ITS 게임웹운영 인원 채용,경기 성남시 분당구,계약직,경력무관,"대학(2,3년)↑",오늘마감(13일 전 등록),https://www.saramin.co.kr/zf_user/jobs/relay/v...,모집부문 및 자격요건\n모집부문 담당업무 상세 내용 인원\n게임웹운영 - NC게임 ...
4,게임개발,(주)네트로,(주)네트로 부문별 신입/경력 채용,대구 수성구,정규직,신입 · 경력,고졸↑,~ 06/30(금)(13일 전 등록),https://www.saramin.co.kr/zf_user/jobs/relay/v...,(주)네트로\n부문별 신입/경력 채용\n모집부분 및 상세내역\n모집부문 담당업무 자...
...,...,...,...,...,...,...,...,...,...,...
381,게임개발,(주)잔디소프트,개발 툴 프로그래머 모집,서울전체,정규직,신입 · 경력,학력무관,상시채용(424일 전 등록),https://www.saramin.co.kr/zf_user/jobs/relay/v...,NaN
382,게임개발,(주)잔디소프트,엔진 개발자 모집,서울전체,정규직,2 ~ 20년,대학교(4년)↑,상시채용(430일 전 등록),https://www.saramin.co.kr/zf_user/jobs/relay/v...,NaN
383,게임개발,(주)모바인어스,유니티기반 기능성 게임 개발자,서울전체,정규직,2 ~ 10년,"대학(2,3년)↑",상시채용(664일 전 등록),https://www.saramin.co.kr/zf_user/jobs/relay/v...,NaN
384,게임개발,(주)앤스시스템,게임 개발자,서울전체,정규직,5 ~ 8년,"대학(2,3년)↑",상시채용(538일 전 등록),https://www.saramin.co.kr/zf_user/jobs/relay/v...,NaN


In [114]:
frame_urls[:10]

['https://www.saramin.co.kr/zf_user/jobs/relay/view-detail?rec_idx=45928447&rec_seq=0&t_category=relay_view&t_content=view_detail&t_ref=&t_ref_content=',
 'https://www.saramin.co.kr/zf_user/jobs/relay/view-detail?rec_idx=45987137&rec_seq=0&t_category=relay_view&t_content=view_detail&t_ref=&t_ref_content=',
 'https://www.saramin.co.kr/zf_user/jobs/relay/view-detail?rec_idx=45936064&rec_seq=0&t_category=relay_view&t_content=view_detail&t_ref=&t_ref_content=',
 'https://www.saramin.co.kr/zf_user/jobs/relay/view-detail?rec_idx=45908179&rec_seq=0&t_category=relay_view&t_content=view_detail&t_ref=&t_ref_content=',
 'https://www.saramin.co.kr/zf_user/jobs/relay/view-detail?rec_idx=45905465&rec_seq=0&t_category=relay_view&t_content=view_detail&t_ref=&t_ref_content=',
 'https://www.saramin.co.kr/zf_user/jobs/relay/view-detail?rec_idx=45991805&rec_seq=0&t_category=relay_view&t_content=view_detail&t_ref=&t_ref_content=',
 'https://www.saramin.co.kr/zf_user/jobs/relay/view-detail?rec_idx=45991675&

In [130]:
df.to_csv('./사람인.csv', index=False)

In [129]:
find_body(df, frame_urls[10:])

100%|██████████| 376/376 [04:38<00:00,  1.35it/s]
